<a href="https://colab.research.google.com/github/RationalTangle/compose_symphony_parser/blob/main/Harmony_v5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Start

In [1]:
!pip install -q condacolab
import condacolab
condacolab.install()

⏬ Downloading https://github.com/jaimergp/miniforge/releases/latest/download/Mambaforge-colab-Linux-x86_64.sh...
📦 Installing...
📌 Adjusting configuration...
🩹 Patching environment...
⏲ Done in 0:00:26
🔁 Restarting kernel...


In [5]:
#@title Install dependencies
!rm -rf compose_symphony_parser
!git clone https://github.com/RationalTangle/compose_symphony_parser.git
!conda env create -f compose_symphony_parser/environment.yml
!pip install -r compose_symphony_parser/requirements.txt > /dev/null

Cloning into 'compose_symphony_parser'...
remote: Enumerating objects: 409, done.
remote: Counting objects: 100% (143/143), done.
remote: Compressing objects: 100% (106/106), done.
remote: Total 409 (delta 66), reused 43 (delta 37), pack-reused 266
Receiving objects: 100% (409/409), 1.30 MiB | 13.83 MiB/s, done.
Resolving deltas: 100% (234/234), done.

CondaValueError: prefix already exists: /usr/local



In [6]:
#@title Inputs
symphony_id = "n8Kiy3tje3CfH25500uO" #@param {type:"string"}
benchmark_ticker = "SPY" #@param {type:"string"}
use_simulated_etf_dataset = True #@param {type:"boolean"}

print(symphony_id, benchmark_ticker, use_simulated_etf_dataset)

n8Kiy3tje3CfH25500uO SPY True


In [7]:
#@title Fetch Data and Compute Allocations
from compose_symphony_parser.lib import symphony_object

symphony = symphony_object.get_symphony(symphony_id)
symphony_name = symphony['fields']['name']['stringValue']
print(symphony_name)
print(f"https://app.composer.trade/symphony/{symphony_id}")

root_node = symphony_object.extract_root_node_from_symphony_response(symphony)

from compose_symphony_parser.lib import traversers, get_backtest_data

tickers = traversers.collect_referenced_assets(root_node)

#
# Get Data
#
import os
try:
  os.mkdir("data")
except FileExistsError:
  pass

closes = get_backtest_data.get_backtest_data(
    tickers.union([benchmark_ticker]), use_simulated_etf_dataset)

#
# Execute logic
#
from compose_symphony_parser.lib import transpilers
allocations, branch_tracker = transpilers.VectorBTTranspiler.execute(
    root_node, closes)

#
# Allocation / Branch Reporting
#
logic_start = branch_tracker.index.min().date()
allocations_possible_start = allocations.index.min().date()

backtest_start = allocations.dropna().index.min().date()
backtest_end = allocations.index.max().date()

print(
    f"Logic can execute from {logic_start} ({len(branch_tracker.index)})")
print(
    f"Allocations can start {allocations_possible_start} ({len(allocations.index)})")
print(f"Start: {backtest_start}")
print(f"End: {backtest_end}")
print()

allocations_aligned = allocations[allocations.index.date >= backtest_start]
branch_tracker_aligned = branch_tracker[branch_tracker.index.date >= backtest_start]

assert len(allocations_aligned) == len(branch_tracker_aligned)

Fetching symphony n8Kiy3tje3CfH25500uO from Composer
V 1.4.1A 'The Cheetah' | Proteus Slime + Dividends l 100-0
https://app.composer.trade/symphony/n8Kiy3tje3CfH25500uO
random_folder : yOJWteTgdL
output_folder : work/yOJWteTgdL
[*********************100%***********************]  59 of 59 completed


<string>:240: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
<string>:241: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
<string>:245: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
<string>:248: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many ti

Logic can execute from 2022-06-24 (156)
Allocations can start 2022-06-24 (156)
Start: 2022-06-24
End: 2023-02-06



In [8]:
#@title Date Limitations
for day, ticker in sorted([(closes[ticker].dropna().index.min().date(), ticker) for ticker in closes.columns], reverse=True):
  print(f"{ticker:<5} {day}")


BITI  2022-06-21
TARK  2022-05-02
UVIX  2022-03-30
SARK  2021-11-09
BITO  2021-10-20
COIN  2021-04-14
ARKX  2021-03-30
SGOV  2020-06-01
JEPI  2020-05-21
HIBL  2019-11-07
ARKK  2014-10-31
ARKG  2014-10-31
ARKW  2014-09-30
ARKQ  2014-09-30
USDU  2013-12-18
CURE  2011-06-15
SPHB  2011-05-05
VIXM  2011-01-04
SPXU  2009-06-25
TECS  2008-12-30
TECL  2008-12-30
YCS   2008-11-25
UCO   2008-11-25
EUO   2008-11-25
ERX   2008-11-19
EPI   2008-02-26
BND   2007-04-10
UUP   2007-03-01
USD   2007-02-01
DBC   2006-02-06
PUI   2005-10-26
GLD   2004-11-18
AGG   2003-09-29
EEM   2003-04-14
UST   2002-07-30
TMV   2002-07-30
BSV   2002-07-30
EFA   2001-08-27
EWZ   2000-07-14
SOXS  2000-06-05
SOXL  2000-06-05
TNA   2000-05-26
TQQQ  1999-03-10
SQQQ  1999-03-10
QQQ   1999-03-10
QLD   1999-03-10
MSTR  1998-06-11
MVV   1995-05-04
SPY   1993-01-29
UVXY  1990-01-02
UPRO  1988-01-04
SSO   1988-01-04
SPXS  1988-01-04
SPXL  1988-01-04
TMF   1986-05-19
TLT   1986-05-19
SHY   1954-07-01
IEF   1954-07-01
BIL   1954-07-

# Branch Reporting

In [9]:
#@title Sanity Checks
branches_by_path = traversers.collect_branches(root_node)
branches_by_leaf_node_id = {
    key.split("/")[-1]: value for key, value in branches_by_path.items()}

# Any days without full allocation?
# Branches involved in days where allocations fail to sum to 1
if len(allocations_aligned[(allocations_aligned.sum(axis=1) - 1).abs() > 0.0001].index):
    print(allocations_aligned[(allocations_aligned.sum(axis=1) - 1).abs() > 0.0001])
    branches_by_failed_allocation_days = branch_tracker_aligned[(
        allocations_aligned.sum(axis=1) - 1).abs() > 0.0001].sum(axis=0)
    branches_with_failed_allocation_days = branches_by_failed_allocation_days[
        branches_by_failed_allocation_days != 0].index.values

    for branch_id in branches_with_failed_allocation_days:
        print(f"  -> id={branch_id} {branches_by_leaf_node_id[branch_id]}")
        print(allocations_aligned[branch_tracker_aligned[branch_id] == 1])

    raise Exception(f"Branches are misbehaving, please contact developer and provide the symphony ({symphony_id})")
else:
    print("All days are 100% allocated.")

               TECL  UCO  USDU  TMF  USD  SPXL  BITI  EFA  SQQQ  YCS  ...  \
2022-06-24  0.03125  0.0   0.0  0.0  0.0   0.0   0.0  0.0   0.0  0.0  ...   
2022-06-27  0.00000  0.0   0.0  0.0  0.0   0.0   0.0  0.0   0.0  0.0  ...   
2022-06-28  0.00000  0.0   0.0  0.0  0.0   0.0   0.0  0.0   0.0  0.0  ...   
2022-06-29  0.12500  0.0   0.0  0.0  0.0   0.0   0.0  0.0   0.0  0.0  ...   
2022-06-30  0.12500  0.0   0.0  0.0  0.0   0.0   0.0  0.0   0.0  0.0  ...   
...             ...  ...   ...  ...  ...   ...   ...  ...   ...  ...  ...   
2023-01-31  0.00000  0.0   0.0  0.0  0.0   0.0   0.0  0.0   0.0  0.0  ...   
2023-02-01  0.12500  0.0   0.0  0.0  0.0   0.0   0.0  0.0   0.0  0.0  ...   
2023-02-02  0.00000  0.0   0.0  0.0  0.0   0.0   0.0  0.0   0.0  0.0  ...   
2023-02-03  0.00000  0.0   0.0  0.0  0.0   0.0   0.0  0.0   0.0  0.0  ...   
2023-02-06  0.00000  0.0   0.0  0.0  0.0   0.0   0.0  0.0   0.0  0.0  ...   

            UVIX  PUI  SGOV  SPXS  BITO  COIN  EPI  SOXS  ARKQ  TARK  
2022

Exception: ignored

In [10]:
#@title Ticker by days (weighted)
ticker_allocation_weights = allocations.mean(
    axis=0).sort_values(ascending=False)
for ticker in ticker_allocation_weights.index:
    print(f"{ticker:<5} {ticker_allocation_weights[ticker]:>5.1%}")

JEPI  86.2%
SARK  30.8%
MSTR  21.2%
BITI   9.6%
SOXL   9.6%
TECL   8.9%
BIL    8.3%
TARK   6.1%
SOXS   4.8%
TQQQ   3.0%
EWZ    2.2%
SQQQ   1.6%
USDU   1.6%
BITO   1.0%
TMF    1.0%
TECS   0.9%
EPI    0.6%
SGOV   0.6%
UPRO   0.6%
TNA    0.6%
ERX    0.3%
ARKQ   0.3%
UVXY   0.0%
UCO    0.0%
EUO    0.0%
SHY    0.0%
YCS    0.0%
GLD    0.0%
COIN   0.0%
SPXU   0.0%
SPXS   0.0%
AGG    0.0%
PUI    0.0%
UVIX   0.0%
ARKX   0.0%
MVV    0.0%
ARKW   0.0%
QLD    0.0%
CURE   0.0%
ARKG   0.0%
EEM    0.0%
TMV    0.0%
DBC    0.0%
ARKK   0.0%
EFA    0.0%
USD    0.0%
SPXL   0.0%


In [11]:
#@title Branch by days activated
backtest_days_count = len(branch_tracker)
branch_enablement = branch_tracker.mean(
    axis=0).sort_values(ascending=False)
for branch_id in branch_enablement.index:
    print(f"{branch_enablement[branch_id]:>5.1%} ({branch_enablement[branch_id] * backtest_days_count:>4.0f} of {backtest_days_count})",
          branches_by_leaf_node_id[branch_id])

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)




 0.0% (   0 of 156) [always] AND RSI(VIXM, 10d) > 75 AND RSI(QQQ, 30d) > 70 AND (not RSI(QQQ, 30d) > 70) AND RSI(QQQ, 30d) < 40 AND (not RSI(QQQ, 30d) < 40) AND RSI(QQQ, 20d) > 75 AND (not RSI(QQQ, 20d) > 75) AND RSI(QQQ, 20d) < 35 AND (not RSI(QQQ, 20d) < 35) AND RSI(QQQ, 10d) > 80 AND (not RSI(QQQ, 10d) > 80) AND RSI(QQQ, 10d) < 30 AND (not RSI(QQQ, 10d) < 30) AND (not RSI(VIXM, 10d) > 75) AND RSI(TQQQ, 10d) > 80 AND (not RSI(TQQQ, 10d) > 80) AND RSI(TQQQ, 10d) < 30 AND (not RSI(TQQQ, 10d) < 30) AND RSI(SPY, 60d) > 50 AND RSI(BND, 20d) > RSI(SPY, 20d) AND (not RSI(BND, 20d) > RSI(SPY, 20d)) AND (not RSI(SPY, 60d) > 50) AND RSI(IEF, 200d) < RSI(TLT, 200d) AND RSI(BND, 20d) > RSI(SPY, 20d) AND (not RSI(BND, 20d) > RSI(SPY, 20d)) AND (not RSI(IEF, 200d) < RSI(TLT, 200d)) AND RSI(SOXL, 10d) < 30 AND (not RSI(SOXL, 10d) < 30) AND RSI(SPY, 60d) > 50 AND RSI(BND, 20d) > RSI(SPY, 20d) AND (not RSI(BND, 20d) > RSI(SPY, 20d)) AND (not RSI(SPY, 60d) > 50) AND RSI(IEF, 200d) < RSI(TLT, 200d) AN

# Transpile Outputs

In [12]:
#@title Human format
from compose_symphony_parser.lib import transpilers

root_node = symphony_object.extract_root_node_from_symphony_response(symphony)
print(transpilers.HumanTextTranspiler.convert_to_string(root_node))

V 1.4.1A 'The Cheetah' | Proteus Slime + Dividends l 100-0
  Weight equally:
    // Proteus Slime 100-0 l Annualized Return 20,709.0%, Sharpe Ratio 10.27, Standard Deviation 54.3%,  Max Drawdown 3.6%
      Weight accordingly:
        // Dividend Area: My preference would be - Pops, Balls, and JEPI l 21May2020 l Can replace JEPI with whatever (PEY/DHS/SPHD, etc. - October 18th 2012 if that group of 3)
          Weight equally:
            // JEPI Double Pops
              Weight equally:
                if
                  (RSI(VIXM, 10d) > 75)
                    Weight equally:
                      // QQQ 321 Pop (BIL)
                        Weight equally:
                          // 30
                            Weight equally:
                              if
                                (RSI(QQQ, 30d) > 70)
                                  SQQQ (max: 8.3%)
                                else
                                  Weight equally:
                              

In [13]:
#@title VectorBT code format
print(transpilers.VectorBTTranspiler.convert_to_string(root_node))



def build_allocations_matrix(closes):
    indicators = pd.DataFrame(index=closes.index)

    indicators['RSI(VIXM, 10d)'] = precompute_indicator(closes['VIXM'], ':relative-strength-index', 10)
    indicators['RSI(QQQ, 30d)'] = precompute_indicator(closes['QQQ'], ':relative-strength-index', 30)
    indicators['RSI(QQQ, 30d)'] = precompute_indicator(closes['QQQ'], ':relative-strength-index', 30)
    indicators['RSI(QQQ, 20d)'] = precompute_indicator(closes['QQQ'], ':relative-strength-index', 20)
    indicators['RSI(QQQ, 20d)'] = precompute_indicator(closes['QQQ'], ':relative-strength-index', 20)
    indicators['RSI(QQQ, 10d)'] = precompute_indicator(closes['QQQ'], ':relative-strength-index', 10)
    indicators['RSI(QQQ, 10d)'] = precompute_indicator(closes['QQQ'], ':relative-strength-index', 10)
    indicators['RSI(TQQQ, 10d)'] = precompute_indicator(closes['TQQQ'], ':relative-strength-index', 10)
    indicators['RSI(TQQQ, 10d)'] = precompute_indicator(closes['TQQQ'], ':relative-streng

# VectorBT

In [15]:
#@title Simulate Portfolio
import vectorbt as vbt

closes_aligned = closes[closes.index.date >=
                            backtest_start].reindex_like(allocations)

portfolio = vbt.Portfolio.from_orders(
    close=closes_aligned,
    size=allocations,
    size_type="targetpercent",
    group_by=True,
    cash_sharing=True,
    call_seq="auto",
    # TODO: rebalancing
    freq='D',
    # TODO: work out Alpaca fees
    fees=0,
)

In [16]:
#@title Quantstats of VectorBT simulation
returns = portfolio.asset_returns()

# Export Quantstats HTML Report
import quantstats
import google.colab

keepcharacters = (' ', '.', '_', '-')
filepath = f"{symphony_name} - Harmony VectorBT.html"
filepath = filepath.replace("%", 'pct ')
filepath = "".join(c for c in filepath if c.isalnum() or c in keepcharacters).rstrip()
quantstats.reports.html(
    returns,
    closes[benchmark_ticker].pct_change().dropna(),
    title=f"{symphony_name} - Harmony VectorBT", output=filepath, download_filename=filepath)

google.colab.files.download(filepath)

findfont: Font family ['Arial'] not found. Falling back to DejaVu Sans.
/usr/local/lib/python3.8/site-packages/quantstats/stats.py:968: FutureWarning: In a future version of pandas all arguments of DataFrame.pivot will be keyword-only.
  returns = returns.pivot('Year', 'Month', 'Returns').fillna(0)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>